### Calcium Imaging Demo Preparation Notebook

**Please Note**: This notebook and demo are NOT intended to be used as learning materials. To gain
a thorough understanding of the DataJoint workflows for calcium imaging, please
see our [`tutorial`](./tutorial.ipynb) notebook.

In [1]:
import os

if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")

import datajoint as dj
import datetime
import matplotlib.pyplot as plt
import numpy as np
from workflow_calcium_imaging.pipeline import subject, session, scan, imaging

[2023-03-07 17:53:28,952][INFO]: Connecting root@fakeservices.datajoint.io:3306
[2023-03-07 17:53:28,959][INFO]: Connected root@fakeservices.datajoint.io:3306


In [2]:
subject.Subject.insert1(
    dict(
        subject='subject1',
        subject_birth_date='2023-01-01',
        sex='U',
    ),
    skip_duplicates=True
)

In [3]:
session.Session.insert1(
    dict(
        subject='subject1',
        session_datetime=datetime.datetime.now()
    )
)
session_key = (session.Session & "subject = 'subject1'").fetch("KEY")[-1]
session.SessionDirectory.insert1(
    dict(
        session_key, 
        session_dir='workspaces/workflow-calcium-imaging/example_data/subject1/session1'
    )
)

In [4]:
scan.Scan.insert1(
    dict(
        session_key,
        scan_id=0,
        acq_software='ScanImage',
    ),
    skip_duplicates=True
)

In [5]:
# Define and insert suite2p parameters

suite2p_params = {
    "look_one_level_down": 0.0,
    "fast_disk": [],
    "delete_bin": False,
    "mesoscan": False,
    "h5py": [],
    "h5py_key": "data",
    "save_path0": [],
    "subfolders": [],
    "functional_chan": 1,
    "tau": 1.0,
    "force_sktiff": True,
    "preclassify": 0.0,
    "save_mat": False,
    "combined": True,
    "aspect": 1.0,
    "do_bidiphase": False,
    "bidiphase": 0.0,
    "do_registration": True,
    "keep_movie_raw": False,
    "nimg_init": 300,
    "batch_size": 500,
    "maxregshift": 0.1,
    "align_by_chan": 1,
    "reg_tif": False,
    "reg_tif_chan2": False,
    "subpixel": 10,
    "smooth_sigma": 1.15,
    "th_badframes": 1.0,
    "pad_fft": False,
    "nonrigid": False,
    "block_size": [128, 128],
    "snr_thresh": 1.2,
    "maxregshiftNR": 5.0,
    "1Preg": False,
    "spatial_hp": 50.0,
    "pre_smooth": 2.0,
    "spatial_taper": 50.0,
    "roidetect": True,
    "sparse_mode": False,
    "diameter": 12,
    "spatial_scale": 0,
    "connected": True,
    "nbinned": 5000,
    "max_iterations": 20,
    "threshold_scaling": 1.0,
    "max_overlap": 0.75,
    "high_pass": 100.0,
    "inner_neuropil_radius": 2,
    "min_neuropil_pixels": 350,
    "allow_overlap": False,
    "chan2_thres": 0.65,
    "baseline": "maximin",
    "win_baseline": 60.0,
    "sig_baseline": 10.0,
    "prctile_baseline": 8.0,
    "neucoeff": 0.7,
    "xrange": np.array([0, 0]),
    "yrange": np.array([0, 0]),
}

imaging.ProcessingParamSet.insert_new_params(
    processing_method="suite2p",
    paramset_desc='Default parameter set for suite2p',
    params=suite2p_params,
    paramset_idx=0,
)

In [6]:
scan_key = (scan.Scan & "subject = 'subject1'").fetch("KEY")[-1]
imaging.ProcessingTask.insert1(
    dict(
        scan_key,
        paramset_idx=0,
        processing_output_dir='workspaces/workflow-calcium-imaging/example_data/subject1/session1',
        task_mode='trigger',
    ),
    skip_duplicates=True
)

In [7]:
populate_settings = {"display_progress": True}
scan.ScanInfo.populate(**populate_settings)
imaging.Processing.populate(**populate_settings)

# Optional Curation Step

processing_key = imaging.Processing.fetch("KEY")[-1]
imaging.Curation.insert1(
    dict(
    processing_key,
    curation_id=0,
    curation_time=datetime.datetime.now(),
    curation_output_dir='workspaces/workflow-calcium-imaging/example_data',
    manual_curation=False,
    )
)

imaging.MotionCorrection.populate(**populate_settings)
imaging.Segmentation.populate(**populate_settings)
imaging.MaskClassification.populate(**populate_settings)
imaging.Fluorescence.populate(**populate_settings)
imaging.Activity.populate(**populate_settings)

Processing:   0%|          | 0/1 [00:00<?, ?it/s]

{'data_path': ['/workspaces/workflow-calcium-imaging/example_data/subject1/session1'], 'tiff_list': ['/workspaces/workflow-calcium-imaging/example_data/subject1/session1/example1.tif']}
FOUND BINARIES AND OPS IN ['/workspaces/workflow-calcium-imaging/example_data/subject1/session1/suite2p/plane0/ops.npy']
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, plane already registered
binary path: /workspaces/workflow-calcium-imaging/example_data/subject1/session1/suite2p/plane0/data.bin
NOTE: Applying builtin classifier at /usr/local/lib/python3.7/site-packages/suite2p/classifiers/classifier.npy
----------- ROI DETECTION
Binning movie in chunks of length 29
Binned movie of size [103,504,506] created in 2.29 sec.
ROIs: 200, cost: 0.2719, time: 37.3356
ROIs: 400, cost: 0.2142, time: 63.2455
ROIs: 547, cost: 0.1907, time: 84.7919
ROIs: 583, cost: 0.1853, time: 98.0249
ROIs: 599, cost: 0.1835, time: 109.7812
ROIs: 599, cost: 0.4262, time: 114.3391
ROIs: 599, c

Activity: 100%|██████████| 1/1 [00:00<00:00,  3.69it/s]


In [12]:
from element_calcium_imaging import imaging_report

imaging_report.ScanLevelReport.populate()
imaging_report.TraceReport.populate()